In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['OCR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('oxygen consumption rate', 1409),
 ('o2 consumption rate', 69),
 ('oculocardiac reflex', 21),
 ('rate of oxygen consumption', 12),
 ('ocrelizumab', 6),
 ('oxygen consumption ratio', 5),
 ('o 2 consumption rate', 5)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-17 03:31:26] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.3/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'o 2 consumption rate': 'MESH:D010101',
  'o2 consumption rate': 'MESH:D010101',
  'ocrelizumab': 'MESH:C533411',
  'oculocardiac reflex': 'MESH:D012024',
  'oxygen consumption rate': 'MESH:D010101',
  'oxygen consumption ratio': 'MESH:D010101',
  'rate of oxygen consumption': 'MESH:D010101'},
 {'MESH:D010101': 'Oxygen Consumption',
  'MESH:C533411': 'ocrelizumab',
  'MESH:D012024': 'Reflex, Oculocardiac'},
 ['MESH:C533411', 'MESH:D010101', 'MESH:D012024']]

In [6]:
grounding_map, names, pos_labels = [{'o 2 consumption rate': 'MESH:D010101',
  'o2 consumption rate': 'MESH:D010101',
  'ocrelizumab': 'MESH:C533411',
  'oculocardiac reflex': 'MESH:D012024',
  'oxygen consumption rate': 'MESH:D010101',
  'oxygen consumption ratio': 'MESH:D010101',
  'rate of oxygen consumption': 'MESH:D010101'},
 {'MESH:D010101': 'Oxygen Consumption',
  'MESH:C533411': 'ocrelizumab',
  'MESH:D012024': 'Reflex, Oculocardiac'},
 ['MESH:C533411', 'MESH:D010101', 'MESH:D012024']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:11243 ', 'HGNC:11243 ', 0)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-17 03:36:03] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-17 03:36:11] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9952995983474319 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'MESH:D012024': 12,
  'MESH:D010101': 856,
  'MESH:C533411': 4},
 'f1': {'mean': 0.9953, 'std': 0.003931},
 'precision': {'mean': 0.994279, 'std': 0.005125},
 'recall': {'mean': 0.996558, 'std': 0.00281},
 'MESH:D012024': {'f1': {'mean': 0.96, 'std': 0.08},
  'pr': {'mean': 0.933333, 'std': 0.133333},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:C533411': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:D010101': {'f1': {'mean': 0.998251, 'std': 0.001428},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.996512, 'std': 0.002848}}}

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for OCR

Produces the disambiguations:
	Oxygen Consumption*	MESH:D010101
	Reflex, Oculocardiac*	MESH:D012024
	ocrelizumab*	MESH:C533411

Class level metrics:
--------------------
Grounding           	Count	F1     
  Oxygen Consumption*	856	0.99825
Reflex, Oculocardiac*	 12	   0.96
         ocrelizumab*	  4	    0.4

Weighted Metrics:
-----------------
	F1 score:	0.9953
	Precision:	0.99428
	Recall:		0.99656

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)